In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import seaborn as sns # for plotting.
import os
import math

In [2]:
filepath = "D:/online dataset csv files preprocessing/merged_u1-50_w001_resampled.csv"
df_original = pd.read_csv(filepath)

In [3]:
df_original

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,924313410000000,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,1
1,924313420000000,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,1
2,924313430000000,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,1
3,924313440000000,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,1
4,924313450000000,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,1
...,...,...,...,...,...,...,...,...
2055043,1368327120000000,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,50
2055044,1368327130000000,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,50
2055045,1368327140000000,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,50
2055046,1368327150000000,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,50


In [4]:
import scipy.stats as stats

Fs = 100
frame_size = Fs*6 
hop_size =int( Fs*1.5)

In [5]:
df_framed = pd.DataFrame(columns=['name','mean_LA(x)', 'mean_LA(y)', 'mean_LA(z)', 'mean_GY(x)', 'mean_GY(y)', 'mean_GY(z)','sd_LA(x)', 'sd_LA(y)', 'sd_LA(z)', 'sd_GY(x)', 'sd_GY(y)', 'sd_GY(z)'])
def get_frames(df, frame_size, hop_size):
    
    last_value = df['name'].iloc[-1]
    for j in range (1,last_value+1): 
        filtered_df = df[df['name'] == j]
        filtered_df = filtered_df.drop('name', axis = 1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            frame = filtered_df[i: i + frame_size]
            
            new_row = [j
                       ,frame['LAx'].mean(),frame['LAy'].mean(),frame['LAz'].mean()
                       ,frame['GYx'].mean(),frame['GYy'].mean(),frame['GYz'].mean()
                       ,frame['LAx'].std(),frame['LAy'].std(),frame['LAz'].std() 
                       ,frame['GYx'].std(),frame['GYy'].std(),frame['GYz'].std()]


            
            df_framed.loc[len(df_framed)] = new_row
            

    return df_framed

In [6]:
df_framed = get_frames(df_original,frame_size,hop_size)
df_framed

,name,mean_LA(x),mean_LA(y),mean_LA(z),mean_GY(x),mean_GY(y),mean_GY(z),sd_LA(x),sd_LA(y),sd_LA(z),sd_GY(x),sd_GY(y),sd_GY(z)
0,1.0,-0.393514,-0.231500,0.259489,-0.136301,0.808596,0.365505,1.607389,2.060070,1.568203,0.718605,1.529494,0.896005
1,1.0,-0.251761,0.021306,0.400439,-0.101273,0.681316,0.286360,2.075153,3.055443,2.730382,1.138002,1.893374,1.028125
2,1.0,0.085337,0.054233,0.570872,0.052196,0.401903,0.113911,2.263569,3.544305,3.857738,1.311719,2.037973,1.039057
3,1.0,0.460568,0.070261,0.548189,0.066615,0.257239,0.155097,3.068828,3.905210,4.735999,1.391489,2.377002,1.031107
4,1.0,0.977162,0.184893,0.420916,0.122411,0.074673,-0.052158,3.646011,3.797331,5.745886,1.701895,2.413996,0.913987
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13513,50.0,-0.953946,-0.955797,1.480089,0.026065,0.071764,-0.107475,3.449477,4.174878,4.467758,1.713124,1.279720,0.823345
13514,50.0,-1.112609,-0.923574,1.790004,-0.037886,0.016173,-0.150160,3.509659,4.391247,4.704499,1.732431,1.365022,0.826021
13515,50.0,-0.837296,-0.930381,1.509779,0.139941,0.116135,-0.067232,3.541378,4.215414,4.575899,1.672514,1.329024,0.804268
13516,50.0,-1.004556,-0.850281,1.637024,-0.023751,0.008108,-0.150729,3.504781,4.129974,4.550813,1.741665,1.362827,0.817100


In [7]:
occurrences = df_framed['name'].value_counts()
print(occurrences.mean())

270.36


In [8]:
X = df_framed.drop('name', axis=1)  # Features (all columns except the target)
y = df_framed['name']  # Labels

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

In [10]:
X_test

,mean_LA(x),mean_LA(y),mean_LA(z),mean_GY(x),mean_GY(y),mean_GY(z),sd_LA(x),sd_LA(y),sd_LA(z),sd_GY(x),sd_GY(y),sd_GY(z)
1037,0.283309,0.244985,-0.028827,0.069292,-0.052283,0.085915,2.550357,3.701928,2.277335,1.143542,1.108720,0.968994
12454,0.027560,-0.133587,-0.080511,-0.065159,0.039048,0.022577,2.779572,2.690868,3.832177,1.485149,1.487502,0.742306
1223,0.386944,0.938543,0.375431,-0.138203,-0.054351,-0.051280,5.433795,3.970670,4.924530,1.396330,3.413953,1.559334
3996,0.419868,-0.471552,-0.104012,-0.127142,0.006869,-0.040241,7.872263,6.663719,6.656298,1.686335,2.627826,2.319782
11613,-0.057131,0.454858,0.479334,-0.100001,0.130166,-0.155761,3.883859,5.049798,6.023952,2.133266,2.185953,0.710668
...,...,...,...,...,...,...,...,...,...,...,...,...
13452,-1.080170,-0.903045,1.452193,-0.027550,0.024543,-0.117008,3.485394,4.242866,4.698736,1.729618,1.281626,0.841778
11341,-0.008512,0.509321,-0.365381,0.042390,-0.031412,0.146490,2.905011,5.127469,2.876506,1.390065,1.235386,1.204719
7548,0.082007,0.390687,0.532448,-0.001286,0.045962,-0.094702,2.653672,4.379451,3.350784,1.621114,1.402731,0.800073
7460,0.067580,0.305718,0.712511,-0.077061,-0.091748,-0.049304,2.613987,4.478969,3.202950,1.570080,1.302956,0.813260


In [11]:
occurrences = y_test.value_counts()
print(occurrences.mean())

81.12


In [12]:
X_train.shape, X_test.shape

((9462, 12), (4056, 12))

In [13]:
y_train.shape, y_test.shape

((9462,), (4056,))

In [14]:
#Create a svm Classifier
model = svm.SVC(C = 4, # reg paramater
                kernel='linear', #kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
               ) # Linear Kernel

#Train the model using the training sets
model.fit(X_train, y_train)


from sklearn import metrics
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9183925049309665


In [15]:
# df_framed.to_csv('MEAN_SD_FRAMED_RESAMPLED_merged_u1-50_w001_resampled.csv', index = False)